importing needed libaries:

In [ ]:

import os, math, timeit
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

define constants

In [ ]:
data_path = "/content/drive/MyDrive/dataset"  #parent dir that has HC_AH & PH_AH

#params
img_size = (224, 224)
batch_size = 4 #the number of samples to process before updating the weights. 4 is good forsmall datasets
epochs = 50
n_runs = 25

all_auc = []

augmenting train data with the transformations we've mentioned.

In [ ]:
#data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255, #normalize pixels from 0-255 to 0-1
    zoom_range=0.2, #random 20% zoom
    width_shift_range=0.1,  # random 10% horizontal shift
    height_shift_range=0.1, # random 10% vertical shift
    brightness_range=[0.8, 1.2],    # random brightness adjustment between 80% and 120%
    horizontal_flip=True, #random horizontal flip
    fill_mode='constant', #when shifting or zooming empty pixels are black
    cval=0.0, #fill value
    validation_split=0.3  #30% for validation + test (20% test, 10% val)
)

we make the test and validation split. Note that you should NEVER augment val and test. we just rescale

In [ ]:
# here we make the 20 and 10 splits from the 30 we set aside for val and test
val_test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.666) # 2/3 of the 30% is validation, 1/3 is test (approx 10% of total for val, 20% for test)

the run loop

In [ ]:
start = timeit.default_timer() #we'll time how long our 25 runs take

for run in range(n_runs):
    print(f"\n========== Run {run+1}/{n_runs} ==========")

    # new split each run
    # train split
    train_generator = train_datagen.flow_from_directory(
        data_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        subset="training",
        classes=["HC_AH", "PD_AH"],
        color_mode="rgb",
        seed=run  #different seed for each run
    )

    # val split
    val_generator = val_test_datagen.flow_from_directory(
        data_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False, #never shuffle val or test
        subset="training", # Use 'training' to get the 10%
        classes=["HC_AH", "PD_AH"],
        color_mode="rgb",
        seed=42  # fixed seed for consistent validation split from the 30%
    )

    # test split
    test_generator = val_test_datagen.flow_from_directory(
        data_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False, #never shuffle val or test
        subset="validation", # Use 'validation' to get the 20%
        classes=["HC_AH", "PD_AH"],
        color_mode="rgb",
        seed=42  #fixed seed for consistent test split from the 30%
    )

    print(f"Train: {train_generator.samples} | Val: {val_generator.samples} | Test: {test_generator.samples}")

    #calculate steps
    steps_per_epoch = math.ceil(train_generator.samples / batch_size) #steps are samples/batch size
    validation_steps = math.ceil(val_generator.samples / batch_size)
    test_steps = math.ceil(test_generator.samples / batch_size)

    # Build model with stronger regularization
    base = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
    base.trainable = False  # freeze base

    #our custom classification head
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.6)(x)  #increased dropout
    x = Dense(128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)  # Added L2 regularization
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)  #additional dropout
    preds = Dense(2, activation="softmax")(x)

    model = Model(inputs=base.input, outputs=preds)

    #..lower the learning rate
    model.compile(
        optimizer=Adam(learning_rate=0.00005),  #reduced learning rate
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.AUC(name='auc')]
    )

    #early stopping and learning rate reduction
    callbacks = [
        EarlyStopping(
            monitor='val_auc',
            patience=10,
            mode='max',
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_auc',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            mode='max',
            verbose=1
        )
    ]

    # train with callbacks
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=validation_steps,
        callbacks=callbacks,
        verbose=1
    )

    # evaluate on test set
    print("\nEvaluating on test set:")
    test_loss, test_accuracy, test_auc_score = model.evaluate(
        test_generator,
        steps=test_steps,
        verbose=1
    )

    print(f"Test Loss: {test_loss:.3f}")
    print(f"Test Accuracy: {test_accuracy:.3f}")
    print(f"Test AUC: {test_auc_score:.3f}")


    # Predictions for ROC AUC (using test set)
    Y_pred_test = model.predict(test_generator, steps=test_steps)
    y_true_test = test_generator.classes[:len(Y_pred_test)]

    if len(np.unique(y_true_test)) > 1:
        auc = roc_auc_score(y_true_test, Y_pred_test[:,1])
        print(f"Test ROC AUC: {auc:.3f}")
        all_auc.append(auc)

        # plot training history for this run

        #accuracy plot
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'Run {run+1} Accuracy')
        plt.legend()

        #loss plot
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'Run {run+1} Loss')
        plt.legend()
        plt.savefig(f"training_history_run{run+1}.png")
        plt.close()
    else:
        print("Skipping ROC AUC calculation for this run due to insufficient classes in the test set.")
        all_auc.append(np.nan) # append nan for this run

    if not all_auc or auc == np.nanmax(all_auc): #checks if this is the best AUC so far. If so, save the model
        model.save(f"MobileNetV2_best.keras")

    # save run results
    pd.DataFrame(all_auc).to_csv("MobileNetV2_AUCs.csv", index=False)


stop = timeit.default_timer()
print("\n========== Summary ==========")
#remvoe nan values before calculating mean and std
valid_auc = [auc for auc in all_auc if not np.isnan(auc)]
print(f"Mean Test AUC: {np.mean(valid_auc):.3f} ± {np.std(valid_auc):.3f}")
print(f"Best Test AUC: {np.nanmax(all_auc):.3f}")
print(f"Runs above 0.7 Test AUC: {sum(auc >= 0.7 for auc in valid_auc)}/{len(valid_auc)}")
print("Total Runtime:", round(stop - start,2), "seconds")


========== Run 1/25 ==========
Found 57 images belonging to 2 classes.
Found 28 images belonging to 2 classes.
Found 53 images belonging to 2 classes.
Train: 57 | Val: 28 | Test: 53
Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 445ms/step - accuracy: 0.4035 - auc: 0.3056 - loss: 1.8931 - val_accuracy: 0.4286 - val_auc: 0.3788 - val_loss: 1.0193 - learning_rate: 5.0000e-05
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 358ms/step - accuracy: 0.4034 - auc: 0.4451 - loss: 1.6806 - val_accuracy: 0.4286 - val_auc: 0.3992 - val_loss: 1.0331 - learning_rate: 5.0000e-05
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 251ms/step - accuracy: 0.4422 - auc: 0.4468 - loss: 1.7392 - val_accuracy: 0.4643 - val_auc: 0.4362 - val_loss: 1.0508 - learning_rate: 5.0000e-05
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 437ms/step - accuracy: 0.4755 - auc: 0.5041 - loss: 1.5439 - val_accuracy: 0.4643 - val_auc: 0.4515 - val_loss: 1.0671 - learning_rate: 5.0000e-05
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 254ms/step - accuracy: 

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 536ms/step - accuracy: 0.6040 - auc: 0.6083 - loss: 1.2261 - val_accuracy: 0.5357 - val_auc: 0.5778 - val_loss: 1.0004 - learning_rate: 5.0000e-05
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 257ms/step - accuracy: 0.5496 - auc: 0.6406 - loss: 1.0652 - val_accuracy: 0.5357 - val_auc: 0.5944 - val_loss: 0.9520 - learning_rate: 5.0000e-05
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 351ms/step - accuracy: 0.5165 - auc: 0.5492 - loss: 1.2131 - val_accuracy: 0.7500 - val_auc: 0.6626 - val_loss: 0.9099 - learning_rate: 5.0000e-05
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 251ms/step - accuracy: 0.3804 - auc: 0.3224 - loss: 1.7119 - val_accuracy: 0.7500 - val_auc: 0.7028 - val_loss: 0.8716 - learning_rate: 5.0000e-05
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 363ms/step - accuracy: 0.4968 - auc: 0.4304 - loss: 1.5631 - val_accuracy: 0.7857 - val_auc: 0.7283 - val_loss: 0.8414 - learning_rate: 5.0000e-05
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 233ms/step - accuracy: 0.5255

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 502ms/step - accuracy: 0.3845 - auc: 0.4112 - loss: 1.8767 - val_accuracy: 0.4643 - val_auc: 0.5536 - val_loss: 0.9831 - learning_rate: 5.0000e-05
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 245ms/step - accuracy: 0.6611 - auc: 0.6824 - loss: 1.1656 - val_accuracy: 0.5357 - val_auc: 0.5555 - val_loss: 0.9802 - learning_rate: 5.0000e-05
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 366ms/step - accuracy: 0.5757 - auc: 0.6812 - loss: 1.1314 - val_accuracy: 0.5357 - val_auc: 0.5491 - val_loss: 0.9604 - learning_rate: 5.0000e-05
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 279ms/step - accuracy: 0.6348 - auc: 0.6297 - loss: 1.3072 - val_accuracy: 0.5714 - val_auc: 0.5714 - val_loss: 0.9493 - learning_rate: 5.0000e-05
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 256ms/step - accuracy: 0.5706 - auc: 0.5284 - loss: 1.2940 - val_accuracy: 0.6071 - val_auc: 0.5714 - val_loss: 0.9409 - learning_rate: 5.0000e-05
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 255ms/step - accuracy: 0.5446

...

15/15 ━━━━━━━━━━━━━━━━━━━━ 21s 924ms/step - accuracy: 0.4837 - auc: 0.5218 - loss: 1.4025 - val_accuracy: 0.5357 - val_auc: 0.5574 - val_loss: 0.9826 - learning_rate: 5.0000e-05
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 13s 403ms/step - accuracy: 0.5149 - auc: 0.4836 - loss: 1.3921 - val_accuracy: 0.5357 - val_auc: 0.5663 - val_loss: 0.9816 - learning_rate: 5.0000e-05
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 267ms/step - accuracy: 0.6688 - auc: 0.6233 - loss: 1.2274 - val_accuracy: 0.5357 - val_auc: 0.5918 - val_loss: 0.9629 - learning_rate: 5.0000e-05
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 259ms/step - accuracy: 0.5538 - auc: 0.5554 - loss: 1.5596 - val_accuracy: 0.5714 - val_auc: 0.5969 - val_loss: 0.9494 - learning_rate: 5.0000e-05
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 386ms/step - accuracy: 0.5781 - auc: 0.6327 - loss: 1.2072 - val_accuracy: 0.5714 - val_auc: 0.5982 - val_loss: 0.9384 - learning_rate: 5.0000e-05
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 253ms/step - accuracy: 0.626

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 418ms/step - accuracy: 0.4766 - auc: 0.4736 - loss: 1.6942 - val_accuracy: 0.5357 - val_auc: 0.6161 - val_loss: 0.9001 - learning_rate: 5.0000e-05
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 361ms/step - accuracy: 0.4578 - auc: 0.4909 - loss: 1.4153 - val_accuracy: 0.6071 - val_auc: 0.6454 - val_loss: 0.8840 - learning_rate: 5.0000e-05
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 253ms/step - accuracy: 0.4716 - auc: 0.5206 - loss: 1.2652 - val_accuracy: 0.6071 - val_auc: 0.6645 - val_loss: 0.8793 - learning_rate: 5.0000e-05
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 362ms/step - accuracy: 0.6536 - auc: 0.7196 - loss: 1.0211 - val_accuracy: 0.6071 - val_auc: 0.6607 - val_loss: 0.8797 - learning_rate: 5.0000e-05
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 267ms/step - accuracy: 0.4447 - auc: 0.4227 - loss: 1.6845 - val_accuracy: 0.5714 - val_auc: 0.6543 - val_loss: 0.8942 - learning_rate: 5.0000e-05
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 420ms/step - accu

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 415ms/step - accuracy: 0.5575 - auc: 0.5787 - loss: 1.4083 - val_accuracy: 0.4643 - val_auc: 0.4094 - val_loss: 1.1325 - learning_rate: 5.0000e-05
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 279ms/step - accuracy: 0.3701 - auc: 0.3323 - loss: 1.7589 - val_accuracy: 0.5000 - val_auc: 0.4145 - val_loss: 1.1140 - learning_rate: 5.0000e-05
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 246ms/step - accuracy: 0.6265 - auc: 0.6156 - loss: 1.6278 - val_accuracy: 0.5000 - val_auc: 0.4145 - val_loss: 1.0888 - learning_rate: 5.0000e-05
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 383ms/step - accuracy: 0.4150 - auc: 0.4840 - loss: 1.5168 - val_accuracy: 0.5000 - val_auc: 0.4171 - val_loss: 1.0738 - learning_rate: 5.0000e-05
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 295ms/step - accuracy: 0.6290 - auc: 0.6061 - loss: 1.2616 - val_accuracy: 0.5000 - val_auc: 0.4324 - val_loss: 1.0519 - learning_rate: 5.0000e-05
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 267ms/step - accur

testing against unseen data

In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the best model
best_model = load_model('/content/MobileNetV2_best.keras')

# Evaluate the model on the test set
test_loss, test_accuracy, test_auc_score = best_model.evaluate(test_generator, steps=test_steps)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test AUC: {test_auc_score}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 150ms/step - accuracy: 0.7187 - auc: 0.8302 - loss: 0.7523
Test Loss: 0.6962056159973145
Test Accuracy: 0.7735849022865295
Test AUC: 0.8672125339508057
